<a href="https://colab.research.google.com/github/basselkassem/easy21/blob/master/environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libs

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# Game configuration

## Episodes number

In [6]:
EPISODES_NUMS = np.array([10**2, 10**3, 10**4,10**5])
print(EPISODES_NUMS)

[   100   1000  10000 100000]


## Maximum score to lose

In [0]:
MAX_VALUE = 21

## Minimum score to lose

In [0]:
MIN_VALUE = 1

## Maximum of the cards values

In [0]:
CARD_VALUE_MAX = 10

## Minimum of the cards values

In [0]:
CARD_VALUE_MIN = 1

## Maximum score for dealer to stick

In [0]:
DEALER_THRESHOLD = 17

## Action space

In [12]:
HIT = 0
STICK = 1
ACTION_SPACE = np.array([HIT, STICK])
print(ACTION_SPACE)

[0 1]


## State Space
The **state** is represented as a 2D array
*   rows represent the dealer first card
*   columns represent the player's score



In [13]:
DEALER_FIRST_CARD_SPACE = np.arange(CARD_VALUE_MIN, CARD_VALUE_MAX + 1)
print(DEALER_FIRST_CARD_SPACE, DEALER_FIRST_CARD_SPACE.shape )
PLAYER_SOCRE = np.arange(MIN_VALUE, MAX_VALUE + 1)
print(PLAYER_SOCRE, PLAYER_SOCRE.shape)
STATE_SPACE_SHAPE = (len(DEALER_FIRST_CARD_SPACE), len(PLAYER_SOCRE),)
print(STATE_SPACE_SHAPE)

[ 1  2  3  4  5  6  7  8  9 10] (10,)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] (21,)
(10, 21)


## Action-State Space
The **Action-State** is represented as a 3D array


*   1th-D represents the dealer first card
*   2nd-D represent the player's score
*   3th-D represent the action



    
    


In [14]:
ACTION_STATE_SPACE_SHAPE = (len(DEALER_FIRST_CARD_SPACE), len(PLAYER_SOCRE), len(ACTION_SPACE))
print(ACTION_STATE_SPACE_SHAPE)

(10, 21, 2)


## Epsilon greedy constant

In [15]:
N0_VALUES = np.array([50, 100, 1000])
print(N0_VALUES)

[  50  100 1000]


#Card Class

In [0]:
class Card:
    def __init__(self, value, color):
        self.value = value
        self.color = color
        
    def display(self):
        print('color:', self.color, 'value:', self.value)
        
    def get_value(self):
        if self.color == 'red':
            return -self.value
        else:
            return self.value

#CardDeck Class

In [0]:
class CardDeck:
    def __init__(self):
        pass
    
    @staticmethod
    def draw():
        red_card_prop = 1 / 3
        color = ''
        if np.random.random() < red_card_prop:
            color = 'red'
        else:
            color = 'black'
        value = np.random.randint(CARD_VALUE_MIN, CARD_VALUE_MAX + 1)
        card = Card(value, color)
        return card
    
    @staticmethod
    def draw_black_card():
        color = 'black'
        value = np.random.randint(CARD_VALUE_MIN, CARD_VALUE_MAX + 1)
        card = Card(value, color)
        return card

#Player Class

In [0]:
class Player:
    def __init__(self):
        self.score = 0
        self.first_card = 0
    
    def set_score(self, score):
        self.score = score
    
    def hit(self):
        new_card = CardDeck.draw()
        self.score += new_card.get_value()

    def is_busted(self):
        busted = False
        if self.score < MIN_VALUE or self.score > MAX_VALUE:
            busted = True
        return busted
            
    def display_status(self):
        print('Score:', self.score)
        print('Busted:', self.is_busted())

# Environment Class

In [0]:
class Environment:

    def __init__(self, mode = 'debug' ):
        self.dealer = Player()
        self.player = Player()
        self.next_state = None
        self.reward = 0
        self.time_step = 0
        self.done = False
        self.mode = mode
    
    def sample_action(self):
        return np.random.randint(HIT, STICK + 1)
        
    def reward_func(self):
        if self.player.is_busted():
            return -1
        if self.dealer.is_busted():
            return 1
        return 0
    
    def is_done(self):
        if self.player.is_busted() or self.dealer.is_busted():
            self.done = True
        else:
            self.done = False
        return self.done
    
    def init(self):
        player_first_card = CardDeck.draw_black_card()
        dealer_first_card = CardDeck.draw_black_card()

        self.dealer.set_score(dealer_first_card.get_value())
        self.dealer.first_card = dealer_first_card.get_value()

        self.player.set_score(player_first_card.get_value())
        self.player.first_card = player_first_card.get_value()
        
        self.next_state = (self.dealer.first_card, self.player.score)

        return (self.next_state, self.reward, self.done)
    
    def dealer_step(self):
        self.dealer.hit()
        self.reward = self.reward_func()
        self.is_done()
    
    def player_step(self):
        self.player.hit()
        self.next_state = (self.dealer.first_card, self.player.score)
        self.reward = self.reward_func()
        self.is_done()
        
    def dealer_policy(self):
        action = None
        dealer_score = self.dealer.score
        if dealer_score >= DEALER_THRESHOLD:
            action = STICK
        else:
            action = HIT
        return action
    
    def handel_two_sticks(self):
        if self.dealer.score > self.player.score:
            self.reward = -1
        elif self.dealer.score < self.player.score:
            self.reward = 1
        else:
            self.reward = 0
    
    def display_info(self, name, action):
        if name == 'Player':
            print('Player: ', action)
            self.player.display_status()
            print('------')
            print('Dealer: not its turn yet')
            self.dealer.display_status()
        else:
            print('Player: Sticked')
            self.player.display_status()
            print('------')
            print('Dealer: ', action)
            self.dealer.display_status()
        print('reward:', self.reward, 'is_done:', self.done)
        print()        
    def re_init(self, dealer_first_card, player_score):
        self.player.score = player_score
        self.dealer.first_card = dealer_first_card
        self.dealer.score = dealer_first_card

    def step(self, state = None, player_action = -1):
        dealer_first_card, player_score = state
        self.re_init(dealer_first_card, player_score)
        if not self.done:
            self.time_step += 1
            if player_action == HIT:
                self.player_step()
                if self.mode == 'debug':
                    self.display_info('Player', player_action)
            elif player_action == STICK:
                while not(self.dealer.is_busted() or self.done):
                    dealer_action = self.dealer_policy()
                    if dealer_action == HIT:
                        self.dealer_step()
                    elif dealer_action == STICK:
                        self.done = True
                        self.handel_two_sticks()
                    else:
                        print('Unknown dealer action')
                    if self.mode == 'debug':
                        self.display_info('Dealer', dealer_action)
            else:
                print('Unkown player action')
        else:
            if self.mode == 'debug':
                print('Game is over')
        return (self.next_state, self.reward, self.done)

In [31]:
env = Environment(mode = 'debug')
next_state, reward, done = env.init()

for i in range(100):
    player_action = env.sample_action()
    next_state, reward, done = env.step(next_state, player_action = 0)
    if done:
        break

Player:  0
Score: 3
Busted: False
------
Dealer: not its turn yet
Score: 1
Busted: False
reward: 0 is_done: False

Player:  0
Score: 9
Busted: False
------
Dealer: not its turn yet
Score: 1
Busted: False
reward: 0 is_done: False

Player:  0
Score: 13
Busted: False
------
Dealer: not its turn yet
Score: 1
Busted: False
reward: 0 is_done: False

Player:  0
Score: 21
Busted: False
------
Dealer: not its turn yet
Score: 1
Busted: False
reward: 0 is_done: False

Player:  0
Score: 27
Busted: True
------
Dealer: not its turn yet
Score: 1
Busted: False
reward: -1 is_done: True

